# node2vec

In [1]:
import pandas as pd
import tqdm as tq
import gensim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
PATH = "/Users/apple/Google Drive/_My Data Analytics Exercise/Exercise/Cytoscape/KEGG enzymes/"

In [45]:
# The original sentence is 
# "L-glutamate catalyses 
# ATP and L-cysteine 
# to produce 
# ADP and phosphate and gamma-L-glutamyl-L-cysteine"
nodes = ['L-glutamate', 'catalyses', 'ATP', 'and', 'L-cysteine', 'to', 'produce', 'ADP', 'phosphate', 'gamma-L-glutamyl-L-cysteine']

In [3]:
df = pd.read_csv(PATH+"RandomWalk.csv")

In [4]:
df.head()

,path
0,"['L-glutamate', 'catalyses', 'L-glutamate', 'c..."
1,"['L-glutamate', 'catalyses', 'L-glutamate', 'c..."
2,"['L-glutamate', 'catalyses', 'L-glutamate', 'c..."
3,"['L-glutamate', 'catalyses', 'ATP', 'catalyses..."
4,"['L-glutamate', 'catalyses', 'ATP', 'L-cystein..."


In [63]:
from tqdm import tqdm
def generate_corpus(df, nodes):
    """extract texts from dataframe into proper corpus"""
    text = list()
    collection = list()
    for i in tqdm(range(len(df))):
        sent = df.loc[i, 'path']
        for t in sent.split(","):
            for node in nodes:
                if node in t:
                    text.append(node)
        collection.append(text)
    return collection

In [64]:
corpus = generate_corpus(df, nodes)

100%|██████████| 10000/10000 [00:00<00:00, 39692.63it/s]


In [55]:
type(corpus)

list

In [29]:
from gensim.models import Word2Vec

In [59]:
import time
start = time.time()
model = Word2Vec(sentences=corpus, window=5, min_count=1, workers=10)
sec = time.time()-start
print ("Processing time: %.4f" %sec)

Processing time: 1044.3459


In [65]:
model.save(PATH+"node2vec.model")

In [67]:
# Similarity comparison
for node in nodes:
    print ("Token = %s" %node)
    print (model.wv.similar_by_word(node))
    print ()

Token = L-glutamate
[('and', 0.587877631187439), ('catalyses', 0.5706859827041626), ('L-cysteine', 0.45024657249450684), ('to', -0.09922177344560623), ('produce', -0.1698339581489563), ('gamma-L-glutamyl-L-cysteine', -0.33798545598983765), ('phosphate', -0.3571850657463074), ('ADP', -0.555553138256073), ('ATP', -0.7430402636528015)]

Token = catalyses
[('L-glutamate', 0.5706859827041626), ('and', 0.14228102564811707), ('L-cysteine', 0.013536369428038597), ('ATP', -0.1659509837627411), ('to', -0.33440345525741577), ('produce', -0.42417845129966736), ('gamma-L-glutamyl-L-cysteine', -0.6173654794692993), ('phosphate', -0.6421977281570435), ('ADP', -0.7165749073028564)]

Token = ATP
[('ADP', 0.09276409447193146), ('gamma-L-glutamyl-L-cysteine', -0.12319786101579666), ('to', -0.1596231907606125), ('phosphate', -0.1645449548959732), ('catalyses', -0.1659509837627411), ('produce', -0.19881166517734528), ('L-cysteine', -0.72438645362854), ('L-glutamate', -0.7430402636528015), ('and', -0.868974